**Author: Alexander Kollmann, 09/2022**

---

**Funktion**

FHIR - Abfragen und Suchen (nativ und mittels fhirclient)

*   einfache Suche über im Standard definierte Such-Parameter (? bzw. &)
*   Verkettete Suche über Referenzen
*   Inkludieren von Referenzierten Ressourcen in das Ergebnis
*   AddOn (z.B. Konvertierung des Suchergebnisses (JSON Bundle) in ein Pandas Dataframe)




---



**Referenzen**

https://learn.microsoft.com/en-us/azure/healthcare-apis/fhir/search-samples

https://cloud.google.com/healthcare-api/docs/how-tos/fhir-advanced-search

https://developer.commure.com/docs/apis/fhir/search-parameters/

https://colab.research.google.com/drive/1OSuqqACnCqw8h67E7DKDMyIGO1C2qbEq?usp=sharing

https://developer.commure.com/docs/apis/fhir/search-parameters/


---


In [1]:
import json
import requests
from collections import OrderedDict
from io import StringIO
from IPython.display import IFrame

In [2]:
# Define Base URL
url = "http://hapi.fhir.org/baseR4/" # Open HAPI FHIR Server
#url = "https://server.fire.ly/R4/" # Open Firely Server
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

In [3]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
IFrame('https://www.hl7.org/fhir/patient.html#search', width=1200, height=500)

# Einfache Suche

In [4]:
# Abfrage nach Name UND Geburtsdatum
# Erwartetes Ergebnis: 1 Treffer

req = url + "Patient?" + "birthdate" + "=" + "2022-12-05" + "&" + "name" + "=" + "WienerNeustadtNJrLm"
#req = url + "Patient?" + "birthdate" + "=" + "2022-12-05"
print(req)
response = requests.request("GET", req, headers=headers)
result = response.json()
patient_id = result['entry'][0]['resource']['id']
print(response.text)

http://hapi.fhir.org/baseR4/Patient?birthdate=2022-12-05&name=WienerNeustadtNJrLm
{
  "resourceType": "Bundle",
  "id": "f0ab84aa-6ed7-4be4-9f5c-5b736a6d0474",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:20.774+00:00"
  },
  "type": "searchset",
  "total": 1,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?birthdate=2022-12-05&name=WienerNeustadtNJrLm"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7185197",
    "resource": {
      "resourceType": "Patient",
      "id": "7185197",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:33:01.605+00:00",
        "source": "#2eopjhS2NavITyoA"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTADTNJRLM </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Date of birth</td><td><span>05 December 2022</span></td></tr><

In [5]:
#print(response.text)
patient_id

'7185197'

In [6]:
# Abfrage der Condition zu einem Patienten

# Teatpatient: MrXXX
#patient_id = str(7168516)
# Get Condition of this patient
req = url + "Condition?" + "patient=" + patient_id
response = requests.request("GET", req, headers=headers)
print("Anzahl: " + str(json.loads(response.text)['total']))
print(response.text)

Anzahl: 2
{
  "resourceType": "Bundle",
  "id": "86d51416-995e-4fa4-ab49-badd5d2f0c59",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:21.775+00:00"
  },
  "type": "searchset",
  "total": 2,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?patient=7185197"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/7185205",
    "resource": {
      "resourceType": "Condition",
      "id": "7185205",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:36:47.234+00:00",
        "source": "#fkx8L0bIRfV87Kw3"
      },
      "code": {
        "coding": [ {
          "system": "http://hl7.org/fhir/sid/icd-10-us",
          "code": "E10.65",
          "display": "Type 1 Diabetes Mellitus with Hyperglycemia"
        } ]
      },
      "subject": {
        "reference": "Patient/7185197"
      }
    },
    "search": {
      "mode": "match"
    }
  }, {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/718

In [7]:
# Search Observations with the term ‘Cholesterol’’ in text description.

req = url + "Observation?_text=Cholesterol"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "e738a77f-4471-4677-9713-6c176918d4a9",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:22.775+00:00"
  },
  "type": "searchset",
  "total": 8,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Observation?_text=Cholesterol"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Observation/10766535",
    "resource": {
      "resourceType": "Observation",
      "id": "10766535",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2023-06-22T17:16:01.534+00:00",
        "source": "#VuSVL37MTcEJHVZG",
        "profile": [ "https://nrces.in/ndhm/fhir/r4/StructureDefinition/Observation" ]
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Narrative with Details</b></p><p><b>id</b>: lab-example-01</p><p><b>status</b>: final</p><p><b>code</b>: Cholesterol <span>(Details : LOINC code '35200-5' = 'Cholesterol [Mass or Moles/volume]

In [8]:
# alle Patienten mit Namen
req = url + "Patient?name=WienerNeustadt&_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))

req = url + "Patient?name=WienerNeustadt"
response = requests.request("GET", req, headers=headers)
print(response.text)

Anzahl der Patienten: 32
{
  "resourceType": "Bundle",
  "id": "bbce6171-f10e-4712-9b24-d8dc71289fe3",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:24.775+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?name=WienerNeustadt"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=bbce6171-f10e-4712-9b24-d8dc71289fe3&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/36126320",
    "resource": {
      "resourceType": "Patient",
      "id": "36126320",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2023-11-20T20:44:12.080+00:00",
        "source": "#PWzRHgrNhOfYbnuR"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTADTAXGPJ </b></div><table class=\"hapiProp

In [9]:
# Anzahl der Patienten
req = url + "Patient?_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

Anzahl der Patienten: 659309
{
  "resourceType": "Bundle",
  "id": "e7d27b15-e858-4895-80f6-8af0f53af9d2",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:25.774+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 659309
}


In [10]:
# Anzahl der Patienten
req = url + "Patient?_total=accurate"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "b0496aea-3f30-44f9-b79e-ff7e3bd60296",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:30.346+00:00"
  },
  "type": "searchset",
  "total": 659309,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_total=accurate"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=b0496aea-3f30-44f9-b79e-ff7e3bd60296&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/592911",
    "resource": {
      "resourceType": "Patient",
      "id": "592911",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2020-01-28T17:42:31.116+00:00",
        "source": "#Nr7G3kFyhLaBhEEZ"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Mr Khan <b>MOAZZAM </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Identi

In [11]:
# Rückgabe von Summary-Paramtern einer Ressource (idF. Patient)
req = url + "Patient?_summary=true"
# Rückgabe von ausgewählten Attributen einer Ressource (idF. Ressource Patient mit den Attributen id und gender)
#req = url + "Patient?_elements=id,gender"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "306c1e71-07f3-4e6d-81be-a0667d94bb33",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:32.030+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_summary=true"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=306c1e71-07f3-4e6d-81be-a0667d94bb33&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/592911",
    "resource": {
      "resourceType": "Patient",
      "id": "592911",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2020-01-28T17:42:31.116+00:00",
        "source": "#Nr7G3kFyhLaBhEEZ",
        "tag": [ {
          "system": "http://terminology.hl7.org/Code

In [12]:
# Anzahl der Patienten ohne GebDat
req = url + "Patient?birthdate:missing&_summary=count"

#Anzahl der Patienten mit Sterbekennzeichen
#req = url + "Patient?deceased:true&_summary=count"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "702ac00b-f225-40f8-95a8-e4be4c20c66b",
  "meta": {
    "lastUpdated": "2023-12-12T06:51:33.030+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 454186
}


In [13]:
# Anzahl der Geschlechter und Abfragedauer
req = url + "Patient" + '?' + 'gender=male' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("male: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')
req = url + "Patient" + '?' + 'gender=female' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("female: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')
req = url + "Patient" + '?' + 'gender:missing=true' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("missing: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')

male: 237527 (1.611628 s)
female: 145835 (0.853859 s)
missing: None (20.503783 s)


In [14]:
# String Search
req = url + 'Patient' + '?' + 'family=Koll' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:exact=MrXXX' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:contains=Mr' + '&_elements=name'
response = requests.request("GET", req, headers=headers)
#print(response.text)

for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))

2376392: Kölle
1709914: Kollege
7167994: Koller
7170339: Koller
7162763: Kolli
7162765: Kolli
7162767: Kolli
7162769: Kolli
7162771: Kolli
7162773: Kolli
7162777: Kolli
7162779: Kolli
7162781: Kolli
7162783: Kolli
7162785: Kolli
7162787: Kolli
7162789: Kolli
7162915: Kolli
7163145: Kolli
7163147: Kolli


In [15]:
IFrame('https://www.hl7.org/fhir/search.html#prefix', width=1200, height=500)

In [16]:
# Date Search
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"

# Patienten mit Geburtsjahr
#req = url + 'Patient' + '?' + 'birthdate=1984'

# Patienten mit Geburtstag zwischen zwei Zeitpunkten
req = url + 'Patient?birthdate=gt2016-01-01&birthdate=lt2019-01-01'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "470f52cb-41f5-4ed5-bba1-3dad79bb963e",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:06.726+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?birthdate=gt2016-01-01&birthdate=lt2019-01-01"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=470f52cb-41f5-4ed5-bba1-3dad79bb963e&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/1727761",
    "resource": {
      "resourceType": "Patient",
      "id": "1727761",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2021-01-05T13:00:21.487+00:00",
        "source": "#fJnbOVkf30U4fL52"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Test Name </div><table class=\"hapiPropertyTable\"><tbody><tr><td>Identi

In [17]:
IFrame('https://build.fhir.org/observation.html#search', width=1200, height=500)

In [18]:
# Range Search

code = "8867-4"

req = url + "/Observation?code=" + code + "&value-quantity=ge50&value-quantity=le200"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "fea4464a-f0e1-4527-9f92-156f60443263",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:09.322+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Observation?code=8867-4&value-quantity=ge50&value-quantity=le200"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=fea4464a-f0e1-4527-9f92-156f60443263&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Observation/30190",
    "resource": {
      "resourceType": "Observation",
      "id": "30190",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2019-09-26T14:34:43.313+00:00",
        "source": "#d0126f9b8578248b"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Heart Rate, 73 bpm</div>"
      },
      "status": "final",
      "code": {
        "co

In [19]:
# Reference search
# Alle Patienten zurückgeben, die einen GP haben und den Namen/ Org des GP

req = url + 'Patient?general-practitioner:missing=false&_elements=id,name,generalPractitioner,managingOrganization'
response = requests.request("GET", req, headers=headers)
print(response.text)


{
  "resourceType": "Bundle",
  "id": "087af324-cc9d-4ca8-a660-4a74e986753a",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:10.322+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_elements=id%2Cname%2CgeneralPractitioner%2CmanagingOrganization&general-practitioner%3Amissing=false"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=087af324-cc9d-4ca8-a660-4a74e986753a&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset&_elements=generalPractitioner,id,managingOrganization,name"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/41189635",
    "resource": {
      "resourceType": "Patient",
      "id": "41189635",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2023-12-11T09:37:50.607+00:00",
        "source": "#QAYa7Oep1CX6FXlU",
        "tag": [ {
          "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",


In [20]:
#req = url + "Patient$export"

In [21]:
# Alle Conditions zu einem Patienten

req = url + 'Condition' + '?' + 'patient=' + url + '/Patient/' + patient_id + '&_elements=code'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "3748bcbc-9004-4467-837d-862d6fe66e2b",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:11.323+00:00"
  },
  "type": "searchset",
  "total": 0,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?_elements=code&patient=http%3A%2F%2Fhapi.fhir.org%2FbaseR4%2F%2FPatient%2F7185197"
  } ]
}


# Chained search

In [22]:
req = url + 'Encounter?subject:Patient.name=WienerNeustadt'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "fafa50fa-5174-4563-8100-044b069631c3",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:18.411+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Encounter?subject%3APatient.name=WienerNeustadt"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=fafa50fa-5174-4563-8100-044b069631c3&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Encounter/7185177",
    "resource": {
      "resourceType": "Encounter",
      "id": "7185177",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:13:28.631+00:00",
        "source": "#i4zSE1cPKjCPzfh8"
      },
      "text": {
        "status": "additional",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Encounter zum Patienten:[{'use': 'official', 'text': 'Fachhochschule WienerNeustadtRUerj', 'family': 'W

In [23]:
req = url + "Condition" + "?" + "subject:Patient.name=WienerNeustadt" + "&" + "code=http://hl7.org/fhir/sid/icd-10-us|E10.65" + "&" + "subject:Patient.gender=female&_summary=count"
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "1bd93bed-7c7e-407a-bbff-00f853f64b75",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:25.680+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 18
}


In [24]:
req = url + 'DiagnosticReport?subject:Patient.name=Alex'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "50a1dc96-3e53-4af8-9773-46aa44cd6499",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:37.655+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/DiagnosticReport?subject%3APatient.name=Alex"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=50a1dc96-3e53-4af8-9773-46aa44cd6499&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/DiagnosticReport/333484fd-4524-47d1-b4a8-89dba919b671",
    "resource": {
      "resourceType": "DiagnosticReport",
      "id": "333484fd-4524-47d1-b4a8-89dba919b671",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2020-03-24T18:00:31.816+00:00",
        "source": "#6SbB93pmUicRlB2M",
        "profile": [ "http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note" ],
        "tag": [ {
          "system": "https:

In [25]:
# Alle Diagnosen von Patienten mit Geburtsjahr 1930
req = url + 'Condition?patient.birthdate=1930'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "8fe694d5-c885-4b36-b9c2-a9706a1de979",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:39.215+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?patient.birthdate=1930"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=8fe694d5-c885-4b36-b9c2-a9706a1de979&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/7053220",
    "resource": {
      "resourceType": "Condition",
      "id": "7053220",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-11-02T16:59:15.853+00:00",
        "source": "#hxRzuOkTaCpmVZ5v"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Onychomycosis</div>"
      },
      "clinicalStatus": {
        "coding": [ {
          "system": "http://terminology.h

# Reverse chained search

In [26]:
# Search for all Patients with Diabetes Mellitus

req = url + 'Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006' + '&_summary=count'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "0c6ede82-0052-4ea7-a02b-c6a180ddd940",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:40.723+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 209
}


In [27]:
# Anzahl der Patienten mit einer Observation "Heart Rate"
# https://loinc.org/8867-4/

code = "8867-4"

req = url + "Patient?_has:Observation:patient:code=" + code + "&_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

Anzahl der Patienten: 6561
{
  "resourceType": "Bundle",
  "id": "05bf814f-d4d7-4593-8934-9965146d7137",
  "meta": {
    "lastUpdated": "2023-12-12T06:52:41.721+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 6561
}


In [28]:
# Alle Patienten zurück geben mit einer gewissen Observation (idF. Heart Rate)
# Wenn es viele Treffer gibt, dann wird "Pageing" aktiviert.

req = url + "Patient?_has:Observation:patient:code=8867-4"

# die ersten 5 Ergebnisse zurückgebe
req = url + "Patient?_has:Observation:patient:code=8867-4&_count=5&_sort=-_lastUpdated"


response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "OperationOutcome",
  "text": {
    "status": "generated",
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">ERROR</td><td>[]</td><td>ERROR: canceling statement due to user request</td></tr></table></div>"
  },
  "issue": [ {
    "severity": "error",
    "code": "processing",
    "diagnostics": "ERROR: canceling statement due to user request"
  } ]
}


# Include

In [29]:
# _include lets you search for resource instances and include in the results other resources referenced by the target resource instances. 

req = url + 'Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "e56d9709-b54f-4933-9118-83115a8904ef",
  "meta": {
    "lastUpdated": "2023-12-12T06:53:20.354+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?_include=Condition%3Apatient&code=http%3A%2F%2Fsnomed.info%2Fsct%7C44054006"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=e56d9709-b54f-4933-9118-83115a8904ef&_getpagesoffset=20&_count=20&_pretty=true&_include=Condition%3Apatient&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/601798",
    "resource": {
      "resourceType": "Condition",
      "id": "601798",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2020-02-07T11:03:45.714+00:00",
        "source": "#Io7DEVTOm0wT6Mkc"
      },
      "clinicalStatus": {
        "coding": [ {
          "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
          "code": 

# Reverse Include

In [30]:
#_revinclude allows you to search for resource instances and include in the results other resources that reference the target resource instances. 
req = url + 'Patient?name=WienerNeustadt&gender=female&birthdate=2022-12-05' + '&' + '_revinclude=Condition:patient'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "85200734-a7cd-4233-8f50-f21863f94812",
  "meta": {
    "lastUpdated": "2023-12-12T06:53:21.354+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_revinclude=Condition%3Apatient&birthdate=2022-12-05&gender=female&name=WienerNeustadt"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=85200734-a7cd-4233-8f50-f21863f94812&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7185197",
    "resource": {
      "resourceType": "Patient",
      "id": "7185197",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:33:01.605+00:00",
        "source": "#2eopjhS2NavITyoA"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTAD

# AddOn

In [31]:
# Daten exportieren, wird nicht von jedem FHIR-Server unterstützt
req = url + '$export' + '?' + '_type=Patient,Condition'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "OperationOutcome",
  "text": {
    "status": "generated",
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">ERROR</td><td>[]</td><td>HAPI-0513: Must request async processing for $export</td></tr></table></div>"
  },
  "issue": [ {
    "severity": "error",
    "code": "processing",
    "diagnostics": "HAPI-0513: Must request async processing for $export"
  } ]
}


In [32]:
# Konfiguration des FHIR-Servers abrufen - wichtig um zu wissen, welche Ressourcen/ Funktionen untersützt werden.
# !! es muss nicht jeder FHIR-Server die gesamte Spezifikation unterstützen !!
req = url + 'metadata'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "CapabilityStatement",
  "id": "0cd355ba-69d5-4b3a-9ac8-df913b315266",
  "name": "RestServer",
  "status": "active",
  "date": "2023-12-12T06:53:16.141+00:00",
  "publisher": "Not provided",
  "copyright": "This server is **Open Source Software**, licensed under the terms of the [Apache Software License 2.0](https://www.apache.org/licenses/LICENSE-2.0).",
  "kind": "instance",
  "instantiates": [ "http://hl7.org/fhir/uv/bulkdata/CapabilityStatement/bulk-data" ],
  "software": {
    "name": "HAPI FHIR Server",
    "version": "6.11.2-SNAPSHOT/89cf98b47e/2023-11-29"
  },
  "implementation": {
    "description": "HAPI FHIR Test/Demo Server R4 Endpoint",
    "url": "https://hapi.fhir.org/baseR4"
  },
  "fhirVersion": "4.0.1",
  "format": [ "application/fhir+xml", "xml", "application/fhir+json", "json", "application/x-turtle", "ttl", "html/json", "html/xml", "html/turtle" ],
  "patchFormat": [ "application/fhir+json", "application/fhir+xml", "application/json-patch+json",

In [33]:
import pandas as pd

# Ergebnis (JSON) in Pandas Dataframe konvertieren
req = url + 'Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006&_elements=id,gender,birthDate'
response = requests.request("GET", req, headers=headers)
#print(response.text)

data = json.loads(response.text)

df = pd.json_normalize(data, 'entry')
df.describe()

,fullUrl,resource.resourceType,resource.id,resource.meta.versionId,resource.meta.lastUpdated,resource.meta.source,resource.meta.tag,resource.gender,resource.birthDate,search.mode,resource.meta.profile
count,20,20,20,20,20,20,20,20,20,20,3
unique,20,1,20,4,20,20,1,2,8,1,1
top,https://hapi.fhir.org/baseR4/Patient/601740,Patient,601740,1,2020-02-07T11:03:45.714+00:00,#Io7DEVTOm0wT6Mkc,[{'system': 'http://terminology.hl7.org/CodeSy...,male,1990-11-24,match,[https://www.fhir.philips.com/4.0/StructureDef...
freq,1,20,1,16,1,1,20,17,7,20,3


In [34]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install fhirclient

Note: you may need to restart the kernel to use updated packages.


In [36]:
import ipywidgets as widgets
from IPython.display import display
#from fhir.resources import Patient, Observation
from fhir.resources.patient import Patient
from fhir.resources.observation import Observation
from fhirclient import client

# Funktion zur Abfrage und Anzeige der Observationen
def fetch_and_display_observations(patient_id, url):
    server = client.FHIRClient(url)
    patient = Patient.read(patient_id, server)
    
    if patient:
        observations = Observation.where(struct={"subject": patient_id}, server=server)
        observation_list.value = "Observations for Patient: {}\n".format(patient.name[0].text)
        for obs in observations:
            observation_list.value += "{}: {}\n".format(obs.code.text, obs.valueQuantity.value)
    else:
        observation_list.value = "Patient not found."

# Erstellen der Widgets
patient_id_input = widgets.Text(placeholder="Enter Patient ID", description="Patient ID:")
fetch_button = widgets.Button(description="Fetch Observations")
observation_list = widgets.HTML(value="")

# Funktion beim Klick auf den "Fetch Observations"-Button ausführen
def on_fetch_button_clicked(b):
    patient_id = patient_id_input.value
    fetch_and_display_observations(patient_id, url)

fetch_button.on_click(on_fetch_button_clicked)

# Anzeigen der Widgets
display(patient_id_input, fetch_button, observation_list)


Text(value='', description='Patient ID:', placeholder='Enter Patient ID')

Button(description='Fetch Observations', style=ButtonStyle())

HTML(value='')